In [ ]:
import numpy as np
import torch
import tdg
from tdg import EffectiveRangeExpansion
from tdg import ReducedTwoBodyA1Hamiltonian as A1
from tdg.Luescher import Zeta2D

import matplotlib.pyplot as plt

In [ ]:
torch.set_printoptions(precision=8)
Z = Zeta2D()

Here's a simple tuning routine that takes the ERE, LegoSphere radii, sites, and a starting guess.

In [ ]:
def tune(ere, radii, nx, start):
    lattice =tdg.Lattice(nx)
    # Construct the two-body Hamiltonian
    H   = A1(lattice, [tdg.LegoSphere(r) for r in radii])
    # Use "inverse Lüscher" to find one energy level for each sphere we need to tune
    energies = ere.target_energies(lattice, H.spheres)
    # and tune H until we find coefficients that reproduce those energies.
    coefficients = H.tuning(
        energies.clone().detach(),
        start=start.clone().detach().requires_grad_(True)
    )
    return coefficients

To go towards the continuum limit we want to hold the scattering fixed

In [ ]:
ere = EffectiveRangeExpansion(torch.tensor([1.0, 0.0]))

and describe what interaction we want to use

In [ ]:
radii = [[0,0]]

Let's tune many discretizations!

In [ ]:
# Fewer will finish faster, obviously!
NX = np.arange(7, 29, 2)

and store computed results in a dictionary.

In [ ]:
C = {
    #  For a starting guess for nx we'll use the results for nx-2.
    #  I have a starting guess for 7, courtesy of Mathematica.
    5: torch.tensor([-5.])
}

In [ ]:
# Warning: this step takes some time!  11 minutes on my laptop.
for nx in NX:
    if nx in C:
        continue
    C[nx] = tune(ere, radii, nx, C[nx-2])
    print(f"{nx=}: {C[nx]}")

We process these coefficients into energy eigenvalues and dimensionless x.

x goes through the Luescher zeta function.

In [ ]:
H = {nx: A1(tdg.Lattice(nx), [tdg.LegoSphere(r) for r in radii]) for nx in NX}
E = {nx: H[nx].eigenenergies(C[nx]) for nx in NX}
x = {nx: E[nx] * nx**2 / (2*torch.pi)**2 for nx in NX}
z = {nx: Z(x[nx]) / torch.pi**2 for nx in NX}

Now we can visualize how well our tuning did!

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))


# First draw the zeta function.
exact = torch.linspace(-5.001, 25.001, 1000)
Z.plot(ax, exact, color='gray')

# and the analytic piece of the ERE
ax.plot(exact, ere.analytic(exact), color='black', linestyle='dashed')

# then, for each spatial discretization
for nx in NX:
    ax.plot(
        x[nx].clone().detach().numpy(),
        z[nx].clone().detach().numpy(),
        linestyle='none', marker='o',
        label=f'{nx=}'
    )

ax.set_xlim([-5,25])
ax.set_ylim([-1,1])
ax.legend();

Looks like the two-body energy levels are converging as expected!

In [ ]:
levels = [1, 2, 3, 4, 5, 6]

fig, axs = plt.subplots(len(levels), 1, figsize=(12, 8 * len(levels)), squeeze=False, sharex=True)
axs = [ax[0] for ax in axs]

for level, ax in zip(levels, axs):
    for nx in NX:
        ax.plot(
            [1/nx**2],
            [x[nx][level].clone().detach().numpy()],
            marker='o'
        )
    ax.set_xlabel(r'$1/N_x^2$')
    ax.set_ylabel(r'$x$')

axs[0].set_xlim([0, 1.1/min(NX)**2]);

The convergence appears to be QUADRATIC with nx!

How does the coefficient approach the continuum?

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

nxInverse=torch.linspace(0, 0.2, 1000)
amplitudeAnswer = (-2*np.pi / torch.log( torch.exp(torch.tensor(ere._gamma)) / nxInverse * ere.a / 2))

ax.plot(nxInverse.detach(), amplitudeAnswer.detach(), color='black')

for nx in NX:
    ax.plot([1/nx], [C[nx].detach().numpy()],
        marker='o'
        )

ax.set_xlabel(r"$1/N_x$")
ax.set_ylabel(r"$C_0$");